[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_2%5B4%5D.pdf)

In [22]:
import logging

In [52]:
logger = logging.getLogger("Algorithm info")
logger.setLevel(logging.INFO)
logger.disabled = True

In [1]:
import numpy as np
import pandas as pd
import math
from common import *

norm = np.linalg.norm
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [2]:
class Point(np.ndarray):
    def __new__(cls, input_array):
        obj = np.array(input_array, dtype=float).view(cls)
        return obj
        
    def set_val(self, index, val):
        self[index] = val
        return self

pt = lambda *vals: Point(vals)

### [Wrapper counting fn. calls](https://stackoverflow.com/questions/1301735/counting-python-method-calls-within-another-method)

Zabava s Python anotacijama umjesto wrappanja u klasu.

In [3]:
def counted(fn):
    def wrapper(*args, **kwargs):
        wrapper.called += 1
        return fn(*args, **kwargs)
    wrapper.called = 0
    wrapper.__name__= fn.__name__
    return wrapper

def counts(fn):
    def wrapper(f, *args, **kwargs):
        if hasattr(f, 'called'):
            f.called = 0
            try:
                return fn(f, *args, **kwargs)
            finally:
                wrapper.calls_to_function = f.called
        else:
            return fn(f, *args, **kwargs)
    wrapper.__name__= fn.__name__
    return wrapper

## Funkcije
Funkcije i pripadne tocke

In [4]:
@counted
def f1(x):
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
f1_x0 = pt(-1.9, 2)
f1_xmin = pt(1, 1)
f1_min = 0

@counted
def f2(x):
    x1, x2 = x
    return (x1 - 4) ** 2 + 4 * (x2 - 2) ** 2
f2_x0 = pt(0.1, 0.3)
f2_xmin = pt(4, 2)
f2_min = 0

@counted
def f3(x):
    return np.sum(np.square(np.arange(1, x.shape[0] + 1) - x))
f3_x0 = pt(0, 0, 0, 0, 0)
f3_xmin = pt(1,2,3,4,5)
f3_min = 0

@counted
def f4(x):
    x1, x2 = x
    return abs((x1 - x2) * (x1 + x2)) + math.sqrt(x1 ** 2 + x2 ** 2)
f4_x0 = pt(5.1, 1.1)
f4_xmin = pt(0, 0)
f4_min = 0

@counted
def f6(x):
    x = np.sum(np.square(x))
    return 0.5 + (math.sin(math.sqrt(x)) ** 2 - 0.5) / (1 + 0.001 * x) ** 2
f6_xmin = 0
f6_min = 0

### [Golden section](http://www.fer.unizg.hr/_download/repository/zlatni_rez.txt)

In [5]:
@counts
def golden_section(f, a, b, e=1e-6, k=0.5*(math.sqrt(5) + 1)):
    hi = b - (b - a) / k
    lo = a + (b - a) / k
    
    f_hi = f(hi)
    f_lo = f(lo)
    
    while abs(b - a) > e:
        if f_hi < f_lo:
            b = lo
            lo = hi
            hi = b - (b - a) / k
            f_lo = f_hi
            f_hi = f(hi)
        else:
            a = hi
            hi = lo
            lo = a + (b - a) / k
            f_hi = f_lo
            f_lo = f(lo)
    
    return a, b

In [6]:
@counted 
def test_gs_fn(x): 
    return (x - 4.99)**2

print(golden_section(test_gs_fn, 0, 5))
print(golden_section.calls_to_function)

(4.989999525693381, 4.990000160114857)
35


### [Unimodal](http://www.fer.unizg.hr/_download/repository/unimodalni.txt)

In [67]:
@counts
def unimodal(f, x0, h=1):
    l, r = x0 - h, x0 + h
    m = x0
    step = 1
    
    fm = f(x0)
    fl = f(l)
    fr = f(r)
    
    if fm < fr and fm < fl:
        return l, r
    elif fm > fr:
        while fm > fr:
            logger.info("l = %s, r = %s, m = %s", l, r, m)
            logger.info("l = %f, r = %f, m = %f", fl, fr, fm)
            l, m, fm = m, r, fr
            r = x0 + h * step
            fr = f(r)
            step *= 2
    else:
        while fm > fl:
            logger.info("l = %s, r = %s, m = %s", l, r, m)
            logger.info("l = %f, r = %f, m = %f", fl, fr, fm)
            r, m, fm = m, l, fl
            l = x0 - h * step
            fl = f(l)
            step *= 2
    return l, r

In [69]:
unimodal(f1, pt(1, 1))

(Point([ 0.,  0.]), Point([ 2.,  2.]))

### Coordinate search

In [9]:
@counts
def coordinate_search(f, x0, eps=1e-6):
    x = Point(x0)
    x_prev = x + 2 * eps
    while norm(x_prev - x) > eps:
        x_prev = Point(x)
        for i in range(len(x)):
            f_in_single_dimension = lambda x_i_scalar_value: \
                f(x.set_val(i, x_i_scalar_value))
            l, r = unimodal(f_in_single_dimension, 0.1, x[i])
            l, r = golden_section(f_in_single_dimension, l, r, eps)
            x[i] = (l + r) / 2
    return x

In [10]:
coordinate_search(f2, f2_x0)

Point([ 3.99999977,  2.00000014])

In [11]:
f2_xmin

Point([ 4.,  2.])

### [Simplex](http://www.fer.unizg.hr/_download/repository/simplex.html)

In [12]:
class InvalidAlgorithmSetupException(Exception):
    def __init__(self, message):
        super(InvalidAlgorithmSetupException, self).__init__(message)
    
    def raiseIf(condition, message = "Invalid algorithm setup"):
        if condition:
            raise InvalidAlgorithmSetupException(message)

In [13]:
@counts
def nelder_mead(
    f, x0, step=1, alpha=1, beta=0.5, gamma=2, sigma=0.5, epsilon=1e-6, 
    max_iter=1000):
    InvalidAlgorithmSetupException.raiseIf(
        alpha <= 0 or gamma <= 1 or sigma <= 0 or sigma > 0.5)
    
    l, h = 0, -1
    x = sorted(simplex_pts(x0, step), key=f)
    xc = np.mean(x[:h], axis=0)
    
    reflect = lambda xc, xh: xc + alpha * (xc - xh)
    expand = lambda xc, xr: xc + gamma * (xr - xc)
    contract = lambda xc, xh: xc - beta * (xc - xh)
    shrink = lambda xl, xi: xl + sigma * (xi - xl)
    
    i = 0
    while norm(x[h] - xc) > epsilon and i < max_iter: 
        logger.info("xc = %s", xc)
        logger.info("f(xc) = %f", f(xc))
        
        xr = reflect(xc, x[h])
        
        fxr = f(xr)
        fxl = f(x[l])
        
        if fxr < fxl:
            xe = expand(xc, x[h])
            x[h] = xe if f(xe) < fxl else xr
        else:
            if all(fxr > f(xj) for xj in x[:h]):
                if fxr < f(x[h]):
                    x[h] = xr
                    
                xk = contract(xc, x[h])
                if f(xk) < f(x[h]):
                    x[h] = xk
                else:
                    shrink_xi = lambda xi: shrink(x[l], xi)
                    x = lmap(shrink_xi, x)
            else:
                x[h] = xr
        
        x = sorted(x, key=f)
        xc = np.mean(x[:h], axis=0)
        i += 1
            
    return x[l]

def simplex_pts(x0, step):
    ret = [x0]
    for i in range(len(x0)):
        xp = Point(x0)
        xp[i] += step
        ret.append(xp)
    return np.array(ret)

In [ ]:
fs = [f1,f2,f3,f4]
x0s = [f1_x0,f2_x0,f3_x0,f4_x0]
xmins = [f1_xmin,f2_xmin,f3_xmin,f4_xmin]

for f, x0, xmin in zip(fs, x0s, xmins):
    nelder_min = nelder_mead(f, x0)
    print(nelder_min, xmin)
    print(np.allclose(nelder_min, xmin))

### [Hooke-Jeeves](http://www.fer.unizg.hr/_download/repository/hj.html)

* x0 - pocetna tocka
* xB - bazna tocka 
* xP - pocetna tocka pretrazivanja
* xN - tocka dobivena pretrazivanjem

In [61]:
logger.disabled = False
logger.info("asdf %s", Point([1]))

INFO:Algorithm info:asdf [ 1.]


In [62]:
@counts
def hooke_jeeves(f, x0, epsilon=1e-6, dx=0.5):
    dx = dx * Point(np.ones_like(x0))
    epsilon = epsilon * Point(np.ones_like(x0))

    xp = Point(x0)
    xb = Point(x0)

    while all(dx > epsilon):
        xn = explore(f, xp, dx)
        fxn, fxb = f(xn), f(xb)
        
        logger.info("xb = %s", xb)
        logger.info("xp = %s", xp)
        logger.info("xn = %s", xn)
        logger.info("f(xn) = %f", fxn)
        logger.info("f(xb) = %f", fxb)
        
        if fxn < fxb:
            xp = 2 * xn - xb
            xb = Point(xn)
        else:
            dx /= 2.0
            xp = Point(xb)
        
    return xp

def explore(f, xp, dx):
    x = Point(xp)
    P = f(x)
    for i in range(len(xp)):
        x[i] = x[i] + dx[i]
        if f(x) > P:
            x[i] = x[i] - 2 * dx[i]
            if f(x) > P:
                x[i] = x[i] + dx[i]
    return x

In [63]:
hooke_jeeves(f1, f1_x0)

INFO:Algorithm info:bazna tocka [-1.9  2. ]
INFO:Algorithm info:pocetna tocka [-1.9  2. ]
INFO:Algorithm info:nova tocka [-1.4  2.5]
INFO:Algorithm info:f(xn) = 34.920000
INFO:Algorithm info:f(xb) = 267.620000
INFO:Algorithm info:bazna tocka [-1.4  2.5]
INFO:Algorithm info:pocetna tocka [-0.9  3. ]
INFO:Algorithm info:nova tocka [-1.4  3.5]
INFO:Algorithm info:f(xn) = 242.920000
INFO:Algorithm info:f(xb) = 34.920000
INFO:Algorithm info:bazna tocka [-1.4  2.5]
INFO:Algorithm info:pocetna tocka [-1.4  2.5]
INFO:Algorithm info:nova tocka [-1.65  2.75]
INFO:Algorithm info:f(xn) = 7.098125
INFO:Algorithm info:f(xb) = 34.920000
INFO:Algorithm info:bazna tocka [-1.65  2.75]
INFO:Algorithm info:pocetna tocka [-1.9  3. ]
INFO:Algorithm info:nova tocka [-1.65  3.25]
INFO:Algorithm info:f(xn) = 34.848125
INFO:Algorithm info:f(xb) = 7.098125
INFO:Algorithm info:bazna tocka [-1.65  2.75]
INFO:Algorithm info:pocetna tocka [-1.65  2.75]
INFO:Algorithm info:nova tocka [-1.65  2.75]
INFO:Algorithm info

INFO:Algorithm info:nova tocka [-1.58457031  2.51855469]
INFO:Algorithm info:f(xn) = 6.685920
INFO:Algorithm info:f(xb) = 6.716167
INFO:Algorithm info:bazna tocka [-1.58457031  2.51855469]
INFO:Algorithm info:pocetna tocka [-1.57871094  2.5       ]
INFO:Algorithm info:nova tocka [-1.57871094  2.49902344]
INFO:Algorithm info:f(xn) = 6.654233
INFO:Algorithm info:f(xb) = 6.685920
INFO:Algorithm info:bazna tocka [-1.57871094  2.49902344]
INFO:Algorithm info:pocetna tocka [-1.57285156  2.47949219]
INFO:Algorithm info:nova tocka [-1.571875    2.47851562]
INFO:Algorithm info:f(xn) = 6.620508
INFO:Algorithm info:f(xb) = 6.654233
INFO:Algorithm info:bazna tocka [-1.571875    2.47851562]
INFO:Algorithm info:pocetna tocka [-1.56503906  2.45800781]
INFO:Algorithm info:nova tocka [-1.56503906  2.45703125]
INFO:Algorithm info:f(xn) = 6.585330
INFO:Algorithm info:f(xb) = 6.620508
INFO:Algorithm info:bazna tocka [-1.56503906  2.45703125]
INFO:Algorithm info:pocetna tocka [-1.55820312  2.43554688]
INFO

INFO:Algorithm info:nova tocka [-1.17929687  1.39746094]
INFO:Algorithm info:f(xn) = 4.753850
INFO:Algorithm info:f(xb) = 4.843707
INFO:Algorithm info:bazna tocka [-1.17929687  1.39746094]
INFO:Algorithm info:pocetna tocka [-1.15878906  1.34863281]
INFO:Algorithm info:nova tocka [-1.1578125   1.34765625]
INFO:Algorithm info:f(xn) = 4.661233
INFO:Algorithm info:f(xb) = 4.753850
INFO:Algorithm info:bazna tocka [-1.1578125   1.34765625]
INFO:Algorithm info:pocetna tocka [-1.13632812  1.29785156]
INFO:Algorithm info:nova tocka [-1.13535156  1.296875  ]
INFO:Algorithm info:f(xn) = 4.565891
INFO:Algorithm info:f(xb) = 4.661233
INFO:Algorithm info:bazna tocka [-1.13535156  1.296875  ]
INFO:Algorithm info:pocetna tocka [-1.11289062  1.24609375]
INFO:Algorithm info:nova tocka [-1.11191406  1.24511719]
INFO:Algorithm info:f(xn) = 4.467862
INFO:Algorithm info:f(xb) = 4.565891
INFO:Algorithm info:bazna tocka [-1.11191406  1.24511719]
INFO:Algorithm info:pocetna tocka [-1.08847656  1.19335938]
INFO

INFO:Algorithm info:nova tocka [-0.31552734  0.07666016]
INFO:Algorithm info:f(xn) = 1.783041
INFO:Algorithm info:f(xb) = 1.814972
INFO:Algorithm info:bazna tocka [-0.31552734  0.07666016]
INFO:Algorithm info:pocetna tocka [-0.31259766  0.07958984]
INFO:Algorithm info:nova tocka [-0.31210937  0.08007812]
INFO:Algorithm info:f(xn) = 1.751678
INFO:Algorithm info:f(xb) = 1.783041
INFO:Algorithm info:bazna tocka [-0.31210937  0.08007812]
INFO:Algorithm info:pocetna tocka [-0.30869141  0.08349609]
INFO:Algorithm info:nova tocka [-0.30820312  0.08398438]
INFO:Algorithm info:f(xn) = 1.723506
INFO:Algorithm info:f(xb) = 1.751678
INFO:Algorithm info:bazna tocka [-0.30820312  0.08398438]
INFO:Algorithm info:pocetna tocka [-0.30429687  0.08789062]
INFO:Algorithm info:nova tocka [-0.30380859  0.08837891]
INFO:Algorithm info:f(xn) = 1.701454
INFO:Algorithm info:f(xb) = 1.723506
INFO:Algorithm info:bazna tocka [-0.30380859  0.08837891]
INFO:Algorithm info:pocetna tocka [-0.29941406  0.09277344]
INFO

INFO:Algorithm info:pocetna tocka [-0.20151367  0.04858398]
INFO:Algorithm info:nova tocka [-0.20126953  0.04882812]
INFO:Algorithm info:f(xn) = 1.449969
INFO:Algorithm info:f(xb) = 1.465830
INFO:Algorithm info:bazna tocka [-0.20126953  0.04882812]
INFO:Algorithm info:pocetna tocka [-0.19467773  0.04614258]
INFO:Algorithm info:nova tocka [-0.19443359  0.04638672]
INFO:Algorithm info:f(xn) = 1.434037
INFO:Algorithm info:f(xb) = 1.449969
INFO:Algorithm info:bazna tocka [-0.19443359  0.04638672]
INFO:Algorithm info:pocetna tocka [-0.18759766  0.04394531]
INFO:Algorithm info:nova tocka [-0.18735352  0.04370117]
INFO:Algorithm info:f(xn) = 1.417204
INFO:Algorithm info:f(xb) = 1.434037
INFO:Algorithm info:bazna tocka [-0.18735352  0.04370117]
INFO:Algorithm info:pocetna tocka [-0.18027344  0.04101562]
INFO:Algorithm info:nova tocka [-0.1800293   0.04077148]
INFO:Algorithm info:f(xn) = 1.399460
INFO:Algorithm info:f(xb) = 1.417204
INFO:Algorithm info:bazna tocka [-0.1800293   0.04077148]
INFO

...

INFO:Algorithm info:pocetna tocka [ 0.79140625  0.62762451]
INFO:Algorithm info:nova tocka [ 0.79143677  0.62765503]
INFO:Algorithm info:f(xn) = 0.043663
INFO:Algorithm info:f(xb) = 0.044237
INFO:Algorithm info:bazna tocka [ 0.79143677  0.62765503]
INFO:Algorithm info:pocetna tocka [ 0.79274902  0.62963867]
INFO:Algorithm info:nova tocka [ 0.79277954  0.62966919]
INFO:Algorithm info:f(xn) = 0.043077
INFO:Algorithm info:f(xb) = 0.043663
INFO:Algorithm info:bazna tocka [ 0.79277954  0.62966919]
INFO:Algorithm info:pocetna tocka [ 0.79412231  0.63168335]
INFO:Algorithm info:nova tocka [ 0.79415283  0.63171387]
INFO:Algorithm info:f(xn) = 0.042480
INFO:Algorithm info:f(xb) = 0.043077
INFO:Algorithm info:bazna tocka [ 0.79415283  0.63171387]
INFO:Algorithm info:pocetna tocka [ 0.79552612  0.63375854]
INFO:Algorithm info:nova tocka [ 0.79555664  0.63378906]
INFO:Algorithm info:f(xn) = 0.041874
INFO:Algorithm info:f(xb) = 0.042480
INFO:Algorithm info:bazna tocka [ 0.79555664  0.63378906]
INFO

INFO:Algorithm info:pocetna tocka [ 0.84301147  0.71020508]
INFO:Algorithm info:nova tocka [ 0.84304199  0.7102356 ]
INFO:Algorithm info:f(xn) = 0.024659
INFO:Algorithm info:f(xb) = 0.025235
INFO:Algorithm info:bazna tocka [ 0.84304199  0.7102356 ]
INFO:Algorithm info:pocetna tocka [ 0.84490356  0.71322632]
INFO:Algorithm info:nova tocka [ 0.84493408  0.71325684]
INFO:Algorithm info:f(xn) = 0.024089
INFO:Algorithm info:f(xb) = 0.024659
INFO:Algorithm info:bazna tocka [ 0.84493408  0.71325684]
INFO:Algorithm info:pocetna tocka [ 0.84682617  0.71627808]
INFO:Algorithm info:nova tocka [ 0.84685669  0.71630859]
INFO:Algorithm info:f(xn) = 0.023526
INFO:Algorithm info:f(xb) = 0.024089
INFO:Algorithm info:bazna tocka [ 0.84685669  0.71630859]
INFO:Algorithm info:pocetna tocka [ 0.8487793   0.71936035]
INFO:Algorithm info:nova tocka [ 0.84874878  0.71939087]
INFO:Algorithm info:f(xn) = 0.022974
INFO:Algorithm info:f(xb) = 0.023526
INFO:Algorithm info:bazna tocka [ 0.84874878  0.71939087]
INFO

INFO:Algorithm info:pocetna tocka [ 0.90914307  0.82601929]
INFO:Algorithm info:nova tocka [ 0.90914307  0.8260498 ]
INFO:Algorithm info:f(xn) = 0.008279
INFO:Algorithm info:f(xb) = 0.008684
INFO:Algorithm info:bazna tocka [ 0.90914307  0.8260498 ]
INFO:Algorithm info:pocetna tocka [ 0.91134033  0.83004761]
INFO:Algorithm info:nova tocka [ 0.91134033  0.83007812]
INFO:Algorithm info:f(xn) = 0.007882
INFO:Algorithm info:f(xb) = 0.008279
INFO:Algorithm info:bazna tocka [ 0.91134033  0.83007812]
INFO:Algorithm info:pocetna tocka [ 0.9135376   0.83410645]
INFO:Algorithm info:nova tocka [ 0.91356812  0.83413696]
INFO:Algorithm info:f(xn) = 0.007493
INFO:Algorithm info:f(xb) = 0.007882
INFO:Algorithm info:bazna tocka [ 0.91356812  0.83413696]
INFO:Algorithm info:pocetna tocka [ 0.9157959  0.8381958]
INFO:Algorithm info:nova tocka [ 0.9157959   0.83822632]
INFO:Algorithm info:f(xn) = 0.007111
INFO:Algorithm info:f(xb) = 0.007493
INFO:Algorithm info:bazna tocka [ 0.9157959   0.83822632]
INFO:A

INFO:Algorithm info:pocetna tocka [ 0.98745117  0.97506714]
INFO:Algorithm info:nova tocka [ 0.98748169  0.97509766]
INFO:Algorithm info:f(xn) = 0.000157
INFO:Algorithm info:f(xb) = 0.000227
INFO:Algorithm info:bazna tocka [ 0.98748169  0.97509766]
INFO:Algorithm info:pocetna tocka [ 0.99001465  0.98010254]
INFO:Algorithm info:nova tocka [ 0.99001465  0.98013306]
INFO:Algorithm info:f(xn) = 0.000100
INFO:Algorithm info:f(xb) = 0.000157
INFO:Algorithm info:bazna tocka [ 0.99001465  0.98013306]
INFO:Algorithm info:pocetna tocka [ 0.99254761  0.98516846]
INFO:Algorithm info:nova tocka [ 0.99257813  0.98519897]
INFO:Algorithm info:f(xn) = 0.000055
INFO:Algorithm info:f(xb) = 0.000100
INFO:Algorithm info:bazna tocka [ 0.99257813  0.98519897]
INFO:Algorithm info:pocetna tocka [ 0.9951416   0.99026489]
INFO:Algorithm info:nova tocka [ 0.9951416   0.99029541]
INFO:Algorithm info:f(xn) = 0.000024
INFO:Algorithm info:f(xb) = 0.000055
INFO:Algorithm info:bazna tocka [ 0.9951416   0.99029541]
INFO

Point([ 1.00000153,  1.00000381])

# Zadaci
## 1.
Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku
pretraživanja postavite točku 10. Primijenite sva tri postupka na rješavanje ove funkcije te ispišite
pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više
udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete
zaključiti? 

In [16]:
data = []

for x0 in [ pt(10), pt(20), pt(30) ]:
    for f in [ hooke_jeeves, nelder_mead, coordinate_search ]:
        result = f(f3, x0)
        data.append([f.__name__, x0, f.calls_to_function, result])

pd.DataFrame(data, columns=['Algorithm', 'x0', 'Num calls', 'Result'])

,Algorithm,x0,Num calls,Result
0,hooke_jeeves,[10.0],132,[1.0]
1,nelder_mead,[10.0],207,[1.0]
2,coordinate_search,[10.0],75,[1.00000006231]
3,hooke_jeeves,[20.0],147,[1.0]
4,nelder_mead,[20.0],257,[1.0]
5,coordinate_search,[20.0],77,[1.00000002298]
6,hooke_jeeves,[30.0],158,[1.0]
7,nelder_mead,[30.0],307,[1.0]
8,coordinate_search,[30.0],78,[1.0000002382]


## 2.
Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim
osima na funkcije 1-4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5). Za
svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj
evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti
iz rezultata?

In [17]:
fs = [f1, f2, f3, f4]
x0s = [f1_x0, f2_x0, f3_x0, f4_x0]

data = []
for search_f in [hooke_jeeves, nelder_mead]:
    for f, x0 in zip(fs, x0s):
        result = search_f(f, x0)
        data.append(
            [search_f.__name__, x0, search_f.calls_to_function, result])

pd.DataFrame(data, columns=['Algorithm', 'x0', 'Num calls', 'Result'])

,Algorithm,x0,Num calls,Result
0,hooke_jeeves,"[-1.9, 2.0]",2952,"[1.00000152588, 1.0000038147]"
1,hooke_jeeves,"[0.1, 0.3]",260,"[3.99999961853, 2.00000076294]"
2,hooke_jeeves,"[0.0, 0.0, 0.0, 0.0, 0.0]",342,"[1.0, 2.0, 3.0, 4.0, 5.0]"
3,hooke_jeeves,"[5.1, 1.1]",150,"[3.1, 3.1]"
4,nelder_mead,"[-1.9, 2.0]",3562,"[0.999999763998, 0.999999487937]"
5,nelder_mead,"[0.1, 0.3]",493,"[3.99999995768, 1.99999972735]"
6,nelder_mead,"[0.0, 0.0, 0.0, 0.0, 0.0]",2774,"[0.999999846924, 1.99999971796, 3.00000003086, 4.00000002405, 4.99999947917]"
7,nelder_mead,"[5.1, 1.1]",712,"[-8.3331812619e-07, 1.51895016504e-07]"


## 3.
Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku
(5, 5). Objasnite rezultate!

In [18]:
hooke_jeeves(f4, pt(5, 5))

Point([ 5.,  5.])

In [19]:
nelder_mead(f4, pt(5, 5))

array([ -8.84859430e-08,  -3.86806220e-07])

## 4.
Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5).
Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz
intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke
minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment.
Što možete zaključiti?

Algoritam se puno bolje ponasa kada za pocetnu tocku uzmemo neku koja je blizu stvarnog minimuma, i to neovisno o koraku koji odaberemo.

In [20]:
data = []
for x0 in [ pt(0.5, 0.5), pt(20, 20) ]:
    for step in range(1, 21):
        result = nelder_mead(f1, x0, step=step)
        data.append([x0, 
                     step, 
                     nelder_mead.calls_to_function, 
                     result, 
                     f1(result)])
        
pd.DataFrame(data, columns=['x0', 'Step', 'Num calls', 'Result', 'Cost'])

,x0,Step,Num calls,Result,Cost
0,"[0.5, 0.5]",1,418,"[1.00000663406, 1.00001278068]",6.777450e-11
1,"[0.5, 0.5]",2,4677,"[0.99999989005, 0.999999747216]",1.202295e-13
2,"[0.5, 0.5]",3,2878,"[1.00000161348, 1.00000318316]",2.795237e-12
3,"[0.5, 0.5]",4,1672,"[0.999999272771, 0.999998478111]",9.835678e-13
4,"[0.5, 0.5]",5,1237,"[1.00000047531, 1.00000095745]",2.305811e-13
5,"[0.5, 0.5]",6,6424,"[0.736712151731, 0.541019639844]",6.961811e-02
6,"[0.5, 0.5]",7,861,"[0.999999873029, 0.999999707377]",1.657410e-13
7,"[0.5, 0.5]",8,1312,"[1.00000088168, 1.00000171537]",1.007768e-12
8,"[0.5, 0.5]",9,1011,"[0.99999903608, 0.99999809265]",9.711259e-13
9,"[0.5, 0.5]",10,707,"[0.999999067494, 0.999998073962]",1.241989e-12


## 5.
Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete
više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost
pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni
minimum ako je nađena vrijednost funkcije cilja manja od 10^-4)

In [21]:
ncalls = 200
threshold = 1e-4

data = []
for i in range(ncalls):
    sign = np.sign(np.random.rand(2) - 0.5)
    random = sign * np.random.rand(2) * 50
    result = hooke_jeeves(f6, pt(*random))
    cost = f6(result)
    
    data.append([random,
                 hooke_jeeves.calls_to_function, 
                 result,
                 cost,
                 cost < threshold ])

df_columns = ['x0', 'Num calls', 'Result', 'Cost', 'Min']
df = pd.DataFrame(data, columns=df_columns)
print('Nasao globalni optimum:', np.any(df['Min']))
df

Nasao globalni optimum: False


,x0,Num calls,Result,Cost,Min
0,"[-43.3147068445, 22.6259394011]",197,"[-41.3127537195, 22.6393194518]",0.45177570046849785,False
1,"[-22.5681587593, -33.768202485]",173,"[-22.9438835518, -33.768202485]",0.42972289286832804,False
2,"[-41.6936238746, -19.3759653427]",219,"[-44.1855977516, -16.3375456192]",0.4517757004684972,False
3,"[17.5427024277, -31.5117719403]",202,"[18.747032872, -29.0117719403]",0.3960979202709787,False
4,"[37.5037027314, -1.9106665289]",194,"[37.5037027314, -3.67721087094]",0.4146681881994054,False
5,"[-35.392298575, 11.9482122962]",246,"[-35.4265125948, 12.8456923072]",0.4146681881994062,False
6,"[-0.100282879989, 32.2270285586]",242,"[3.29245736964, 31.2270285586]",0.37329064963265834,False
7,"[20.0832740704, 1.18231353651]",204,"[18.5832740704, 3.07308921705]",0.227690138520178,False
8,"[39.9555857873, -34.7459645206]",242,"[42.0574687219, -32.8938374454]",0.46629508689710913,False
9,"[9.27174384212, -40.6769015035]",179,"[8.51797110653, -39.9269015035]",0.42972289286832915,False


# Matlab
* `fminbnd` (1 var)
* `fminsearch` (vise var; simplex po N&M)
* `fminunc` (vise var; razliciti alg.)
* `fmincon` (vise var, uz ogranicenja)